In [ ]:
import math
import random
import logging
import time
import requests
from typing import Any, Dict, List, Optional, Tuple
import os
import json
import uuid
import concurrent.futures
from tqdm import tqdm
from pydantic import BaseModel, Field

In [ ]:
class CellData(BaseModel):
    owner: Optional[str]
    entity: str
    has_moved: bool


class FieldData(BaseModel):
    height: int
    width: int
    cells: Dict[str, CellData]
    territories: List[Dict[str, List[str]]]


class TerritoryData(BaseModel):
    owner: str
    funds: int
    tiles: List[List[int]]


class GameState(BaseModel):
    players: List[str]
    current_player_index: int
    field_data: FieldData
    territories_data: List[TerritoryData]
    is_game_over: bool = False
    winner: Optional[str] = None
    last_action: Optional[str] = None


class Action(BaseModel):
    action_id: int
    action_type: str
    params: Optional[Dict[str, Any]] = None
    description: Optional[str] = None


class ActionRequest(BaseModel):
    state: GameState
    action_id: Optional[int] = None


class ApplyActionResponse(BaseModel):
    state: GameState
    is_game_over: bool
    winner: Optional[str]

In [ ]:
class GameApi:
    """
    Синхронный клиент для взаимодействия с API игры
    """

    def __init__(self, base_url: str, timeout: int = 30):
        self.base = base_url.rstrip("/")
        self.session = requests.Session()
        self.session.timeout = timeout
        self.query_count = 0
        logging.basicConfig(
            filename="mcst_api_sync.log",
            format="%(asctime)s %(levelname)s %(message)s",
            level=logging.INFO,
        )

    def _inc(self):
        self.query_count += 1

    def generate_state(self, num_players: int = 2, randomize: bool = False) -> GameState:
        self._inc()
        url = f"{self.base}/generate_state"
        r = self.session.post(
            url,
            params={"num_players": num_players, "random": str(randomize).lower()},
        )
        r.raise_for_status()
        return GameState.model_validate(r.json())

    def get_actions(self, state: GameState) -> List[Action]:
        self._inc()
        url = f"{self.base}/get_actions"
        r = self.session.post(url, json=state.model_dump())
        r.raise_for_status()
        return [Action.model_validate(o) for o in r.json()]

    def apply_action(self, state: GameState, action: Action) -> ApplyActionResponse:
        self._inc()
        url = f"{self.base}/apply_action"
        req = ActionRequest(state=state, action_id=action.action_id)
        r = self.session.post(url, json=req.model_dump())
        r.raise_for_status()
        return ApplyActionResponse.model_validate(r.json())

    @staticmethod
    def is_terminal(state: GameState) -> bool:
        return state.is_game_over

    @staticmethod
    def current_player(state: GameState) -> str:
        return state.players[state.current_player_index]

    @staticmethod
    def evaluate_state(state: GameState) -> float:
        """Эвристическая оценка состояния"""
        cells = state.field_data.cells
        total = len(cells)
        cur = state.players[state.current_player_index]
        opp = state.players[1 - state.current_player_index]

        # Подсчет количества захваченных клеток для каждого игрока
        cnt_cur = sum(1 for c in cells.values() if c.owner == cur)
        cnt_opp = sum(1 for c in cells.values() if c.owner == opp)
        diff = cnt_cur / total
        return max(-1.0, min(1.0, diff))


class MCTSNode:
    def __init__(self, state: GameState, parent: Optional["MCTSNode"] = None, action: Optional[Action] = None):
        self.state = state
        self.parent = parent
        self.action = action
        self.children: List["MCTSNode"] = []
        self.visits = 0
        self.value = 0.0
        self.untried_actions: Optional[List[Action]] = None

    def is_fully_expanded(self) -> bool:
        return self.untried_actions is not None and not self.untried_actions

    def best_uct_child(self, c: float) -> "MCTSNode":

        scores = [
            (child.value / child.visits) + c * math.sqrt(math.log(self.visits) / child.visits)
            for child in self.children
            if child.visits > 0
        ]
        if not scores:
            return random.choice(self.children) if self.children else None
        idx = scores.index(max(scores))
        return self.children[idx]

    def find_child_by_action(self, action_id: int) -> Optional["MCTSNode"]:
        """Поиск дочернего узла по ID действия"""
        for child in self.children:
            if child.action and child.action.action_id == action_id:
                return child
        return None


class MCTSPlayer:
    def __init__(
        self, player_id: str, game_api: GameApi, c: float = math.sqrt(2), max_depth: int = 200, workers: int = 1
    ):
        self.player_id = player_id
        self.game = game_api
        self.c = c
        self.max_depth = max_depth
        self.root = None
        self.workers = workers

    def initialize(self, state: GameState):
        """Инициализирует корневой узел дерева"""
        self.root = MCTSNode(state)
        self.root.untried_actions = self.game.get_actions(state)

    def _select(self, node: MCTSNode) -> MCTSNode:
        """Выбор узла для расширения"""
        while not self.game.is_terminal(node.state) and node.is_fully_expanded():
            child = node.best_uct_child(self.c)
            if not child:
                break
            node = child
        return node

    def _expand(self, node: MCTSNode) -> Optional[MCTSNode]:
        """Расширение узла новым дочерним узлом"""
        if not node.untried_actions:
            if not node.children:
                # Загружаем доступные действия
                node.untried_actions = self.game.get_actions(node.state)
                if not node.untried_actions:
                    return None
            else:
                return None

        action = node.untried_actions.pop()
        res = self.game.apply_action(node.state, action)
        child = MCTSNode(res.state, parent=node, action=action)
        child.untried_actions = self.game.get_actions(res.state)  # Сразу загружаем действия
        node.children.append(child)
        return child

    def _simulate(self, node: MCTSNode) -> float:
        """Симуляция из текущего узла"""
        state = node.state
        depth = 0

        while depth < self.max_depth and not self.game.is_terminal(state):
            actions = self.game.get_actions(state)
            if not actions:
                break

            action = random.choice(actions)
            res = self.game.apply_action(state, action)
            state = res.state
            depth += 1

        if self.game.is_terminal(state):
            return 1.0 if state.winner == self.player_id else -1.0

        # Эвристическая оценка для состояний без победы/поражения
        return GameApi.evaluate_state(state)

    def _backpropagate(self, node: MCTSNode, reward: float):
        """Обратное распространение результата симуляции"""
        while node:
            node.visits += 1
            node_player = self.game.current_player(node.state)
            # Инвертируем значение для узлов противника
            node.value += reward if node_player == self.player_id else -reward
            node = node.parent

    def _run_iteration(self):
        """Выполняет одну итерацию MCTS"""
        # 1. Выбор
        node = self._select(self.root)

        # 2. Расширение
        if not self.game.is_terminal(node.state):
            new_node = self._expand(node)
            if new_node:
                node = new_node

        # 3. Симуляция
        reward = self._simulate(node)

        # 4. Распространение
        self._backpropagate(node, reward)

    def search(self, iterations: int, show_progress: bool = False, desc: str = "MCTS") -> Action:
        """Выполняет поиск лучшего хода"""
        if not self.root:
            raise ValueError("MCTS дерево не инициализировано")

        # Проверяем, что у корня есть untried_actions
        if self.root.untried_actions is None:
            self.root.untried_actions = self.game.get_actions(self.root.state)

        # Если доступно только одно действие, сразу его возвращаем
        if len(self.root.untried_actions) == 1:
            return self.root.untried_actions[0]

        if show_progress:
            pbar = tqdm(total=iterations, desc=desc, unit="iter")
            start_time = time.time()
            start_queries = self.game.query_count

        if self.workers > 1:
            iterations_per_worker = iterations // self.workers
            remaining = iterations % self.workers  # можно выкинуть

            # Так как боттлнек в игровом движке (который дергается через API), то можно запускать несколько потоков
            with concurrent.futures.ThreadPoolExecutor(max_workers=self.workers) as executor:
                futures = []

                for i in range(iterations_per_worker):
                    futures.append(executor.submit(self._run_iteration))

                for i, future in enumerate(concurrent.futures.as_completed(futures)):
                    if show_progress:
                        pbar.update(1)
        else:
            for i in range(iterations):
                self._run_iteration()

                if show_progress:
                    elapsed = time.time() - start_time
                    queries_done = self.game.query_count - start_queries
                    qps = queries_done / elapsed if elapsed > 0 else 0
                    pbar.set_postfix({"iter": i + 1, "queries": queries_done, "QPS": f"{qps:.1f}"})
                    pbar.update(1)

        if show_progress:
            pbar.close()

        print(f"Root has {len(self.root.children)} children after search")

        best_child = None
        best_visits = -1

        for child in self.root.children:
            if child.visits > best_visits:
                best_visits = child.visits
                best_child = child

        if not best_child:
            if not self.root.untried_actions:
                self.root.untried_actions = self.game.get_actions(self.root.state)
            if self.root.untried_actions:
                return random.choice(self.root.untried_actions)
            else:
                raise ValueError("No valid actions found!")

        return best_child.action

    def update_root(self, action_id: int, new_state: GameState) -> bool:
        """
        Обновляет корень дерева после выполнения хода.
        Возвращает True если успешно обновили корень, иначе False.
        """
        if not self.root or not self.root.children:
            self.initialize(new_state)
            return False

        child = self.root.find_child_by_action(action_id)

        if child:
            # Если нашли подходящий узел, делаем его новым корнем
            child.parent = None  # Отсоединяем от родителя
            self.root = child
            # Проверяем, есть ли у корня доступные действия
            if self.root.untried_actions is None:
                self.root.untried_actions = self.game.get_actions(self.root.state)
            return True
        else:
            self.initialize(new_state)
            return False

In [ ]:
def simulate_game(
    api: GameApi,
    max_moves: int = 50,
    mcts_iters: int = 500,
    c: float = 1.4,
    max_depth: int = 200,
    workers: int = 1,
) -> None:
    """
    Симуляция игры двух MCTS-игроков
    """
    os.makedirs("logs", exist_ok=True)
    session_id = uuid.uuid4().hex
    date_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
    log_path = os.path.join("logs", f"{session_id}_{date_str}.json")

    state = api.generate_state(num_players=2, randomize=False)

    # Записываем начальное состояние в лог
    with open(log_path, "a", encoding="utf-8") as log_file:
        log_file.write(json.dumps(state.model_dump(), ensure_ascii=False) + "\n")

    player1 = MCTSPlayer(player_id=state.players[0], game_api=api, c=c, max_depth=max_depth, workers=workers)
    player2 = MCTSPlayer(player_id=state.players[1], game_api=api, c=c, max_depth=max_depth, workers=workers)

    player1.initialize(state)
    player2.initialize(state)

    # История ходов
    history = []

    # Основной цикл игры
    for move_no in tqdm(range(1, max_moves + 1), desc="Game moves"):
        # Определяем активного игрока
        current_player_idx = state.current_player_index
        current_player_id = state.players[current_player_idx]

        # Выбираем активного игрока
        active_player = player1 if current_player_id == player1.player_id else player2

        print(f"\nMove {move_no} - Player {current_player_id}")

        # Поиск лучшего хода
        action = active_player.search(
            iterations=mcts_iters, show_progress=True, desc=f"Move {move_no} (Player {current_player_id})"
        )

        print(f"Selected action: {action.action_id} ({action.action_type})")

        response = api.apply_action(state, action)
        new_state = response.state

        history.append((move_no, current_player_id, action.action_id))

        print(f"Player1 размер дерева: {len(player1.root.children) if player1.root else 0}")
        print(f"Player2 размер дерева: {len(player2.root.children) if player2.root else 0}")

        player1_updated = player1.update_root(action.action_id, new_state)
        player2_updated = player2.update_root(action.action_id, new_state)

        print(f"player1_updated: {player1_updated}, player2_updated: {player2_updated}")

        state = new_state

        with open(log_path, "a", encoding="utf-8") as log_file:
            log_file.write(json.dumps(state.model_dump(), ensure_ascii=False) + "\n")

        if response.is_game_over:
            break

    winner = state.winner
    print(f"{move_no} ходов, победил {winner}")

In [7]:
api = GameApi("http://localhost:8080")

simulate_game(
    api,
    max_moves=500,      # Максимальное количество ходов в игре
    mcts_iters=20,     # Количество итераций MCTS при расчете хода
    c=1.4,             
    max_depth=6,       # Отсечка глубины симуляции
    workers=8,  
)

Game moves:   0%|          | 0/500 [00:00<?, ?it/s]


Move 1 - Player player_0





Game moves:   0%|          | 1/500 [00:01<14:20,  1.72s/it]

Root has 2 children after search
Selected action: 4 (spawn_unit)
Player1 размер дерева: 2
Player2 размер дерева: 0
player1_updated: True, player2_updated: False

Move 2 - Player player_0
Selected action: 0 (end_turn)


Game moves:   0%|          | 2/500 [00:02<07:45,  1.07it/s]

...

player1_updated: True, player2_updated: False

Move 15 - Player player_0
Selected action: 0 (end_turn)
Player1 размер дерева: 0
Player2 размер дерева: 0


Game moves:   3%|▎         | 15/500 [00:12<06:13,  1.30it/s]

player1_updated: False, player2_updated: False

Move 16 - Player player_1



Game moves:   3%|▎         | 15/500 [00:13<07:22,  1.10it/s]


KeyboardInterrupt: 